In [311]:
import pandas as pd
import numpy as np
from datetime import datetime
import re
import math

In [312]:
raw = pd.read_csv('./small_raw.csv')
raw.head()

,Unnamed: 0,state_code,latitude,longitude,zip_code,id,city,Unnamed: 6,name,labels,...,object_id,has_VC,has_angel,has_roundA,has_roundB,has_roundC,has_roundD,avg_participants,is_top500,status
0,1005,CA,42.358880,-71.056820,92101,c:6669,San Diego,NaN,Bandsintown,1,...,c:6669,0,1,0,0,0,0,1.0000,0,acquired
1,204,CA,37.238916,-121.973718,95032,c:16283,Los Gatos,NaN,TriCipher,1,...,c:16283,1,0,0,1,1,1,4.7500,1,acquired
2,1001,CA,32.901049,-117.192656,92121,c:65620,San Diego,San Diego CA 92121,Plixi,1,...,c:65620,0,0,1,0,0,0,4.0000,1,acquired
3,738,CA,37.320309,-122.050040,95014,c:42668,Cupertino,Cupertino CA 95014,Solidcore Systems,1,...,c:42668,0,0,0,1,1,1,3.3333,1,acquired
4,1002,CA,37.779281,-122.419236,94105,c:65806,San Francisco,San Francisco CA 94105,Inhale Digital,0,...,c:65806,1,1,0,0,0,0,1.0000,1,closed


In [313]:
raw.drop(labels = "is_othercategory", axis = 1, inplace = True)
raw.drop(labels = "milestones", axis = 1, inplace = True)
raw.drop(labels = "is_software", axis = 1, inplace = True)
raw.drop(labels = "Unnamed: 0", axis = 1, inplace = True)
raw.drop(labels = "latitude", axis = 1, inplace = True)
raw.drop(labels = "longitude", axis = 1, inplace = True)
raw.drop(labels = "zip_code", axis = 1, inplace = True)
raw.drop(labels = "id", axis = 1, inplace = True)
raw.drop(labels = "Unnamed: 6", axis = 1, inplace = True)
raw.drop(labels = "closed_at", axis = 1, inplace = True)
raw.drop(labels = "age_first_milestone_year", axis = 1, inplace = True)
raw.drop(labels = "age_last_milestone_year", axis = 1, inplace = True)
raw.drop(labels = "relationships", axis = 1, inplace = True)
raw.drop(labels = "state_code.1", axis = 1, inplace = True)
raw.drop(labels = "is_CA", axis = 1, inplace = True)
raw.drop(labels = "is_NY", axis = 1, inplace = True)
raw.drop(labels = "is_MA", axis = 1, inplace = True)
raw.drop(labels = "is_TX", axis = 1, inplace = True)
raw.drop(labels = "is_otherstate", axis = 1, inplace = True)
raw.drop(labels = "is_web", axis = 1, inplace = True)
raw.drop(labels = "is_mobile", axis = 1, inplace = True)
raw.drop(labels = "is_enterprise", axis = 1, inplace = True)
raw.drop(labels = "is_advertising", axis = 1, inplace = True)
raw.drop(labels = "is_gamesvideo", axis = 1, inplace = True)
raw.drop(labels = "is_ecommerce", axis = 1, inplace = True)
raw.drop(labels = "is_biotech", axis = 1, inplace = True)
raw.drop(labels = "is_consulting", axis = 1, inplace = True)
raw.drop(labels = "object_id", axis = 1, inplace = True)
raw.drop(labels = "avg_participants", axis = 1, inplace = True)
raw.drop(labels = "is_top500", axis = 1, inplace = True)
raw.drop(labels = "status", axis = 1, inplace = True)
raw.drop(labels = "first_funding_at", axis = 1, inplace = True)
raw.drop(labels = "last_funding_at", axis = 1, inplace = True)

raw.head()

,state_code,city,name,labels,founded_at,age_first_funding_year,age_last_funding_year,funding_rounds,funding_total_usd,category_code,has_VC,has_angel,has_roundA,has_roundB,has_roundC,has_roundD
0,CA,San Diego,Bandsintown,1,1/1/2007,2.2493,3.0027,3,375000,music,0,1,0,0,0,0
1,CA,Los Gatos,TriCipher,1,1/1/2000,5.1260,9.9973,4,40100000,enterprise,1,0,0,1,1,1
2,CA,San Diego,Plixi,1,3/18/2009,1.0329,1.0329,1,2600000,web,0,0,1,0,0,0
3,CA,Cupertino,Solidcore Systems,1,1/1/2002,3.1315,5.3151,3,40000000,software,0,0,0,1,1,1
4,CA,San Francisco,Inhale Digital,0,8/1/2010,0.0000,1.6685,2,1300000,games_video,1,1,0,0,0,0


In [314]:
raw = raw.rename(columns={'name':'Startup_name', 'state_code': 'State_code','labels': 'Status', 'category_code': 'Category', 'founded_at': 'Founded_at',
                          'age_last_funding_year': 'Age_last_funding_year', 'age_first_funding_year': 'Age_first_funding_year',
                          'funding_total_usd': 'Funding_total_usd', 'has_VC': 'Has_VC','has_angel': 'Has_angel',
                         'has_roundA':'Has_roundA', 'has_roundB': 'Has_roundB', 'has_roundC': 'Has_roundC', 'has_roundD': 'Has_roundD',
                         'funding_rounds': 'Funding_rounds'})

In [315]:
clean = raw.copy()
city = clean['city'].value_counts()
city = city.reset_index()
focus_city = city['index'][:14]
raw_city = clean['city'].tolist()
for l in range(len(clean['city'])):
    if raw_city[l] not in focus_city.tolist():
        clean.drop(labels = l, axis = 0, inplace = True)
clean = clean.reset_index()
clean.drop(labels = 'index', axis = 1, inplace = True)
clean.head()

,State_code,city,Startup_name,Status,Founded_at,Age_first_funding_year,Age_last_funding_year,Funding_rounds,Funding_total_usd,Category,Has_VC,Has_angel,Has_roundA,Has_roundB,Has_roundC,Has_roundD
0,CA,San Diego,Bandsintown,1,1/1/2007,2.2493,3.0027,3,375000,music,0,1,0,0,0,0
1,CA,San Diego,Plixi,1,3/18/2009,1.0329,1.0329,1,2600000,web,0,0,1,0,0,0
2,CA,San Francisco,Inhale Digital,0,8/1/2010,0.0000,1.6685,2,1300000,games_video,1,1,0,0,0,0
3,CA,Mountain View,Matisse Networks,0,1/1/2002,4.5452,4.5452,1,7500000,network_hosting,0,0,0,1,0,0
4,CA,Mountain View,RingCube Technologies,1,1/1/2005,1.7205,5.2110,3,26000000,software,1,0,1,1,0,0


In [316]:
ver2 = clean.copy()

In [317]:
ver2['Founded_month'] = ""
for l in range(len(ver2['Founded_at'])):
    ver2['Founded_month'][l] = re.sub(r'/.+', "", ver2['Founded_at'][l])
ver2

C:\Users\USER\AppData\Local\Temp/ipykernel_11988/46680413.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ver2['Founded_month'][l] = re.sub(r'/.+', "", ver2['Founded_at'][l])


,State_code,city,Startup_name,Status,Founded_at,Age_first_funding_year,Age_last_funding_year,Funding_rounds,Funding_total_usd,Category,Has_VC,Has_angel,Has_roundA,Has_roundB,Has_roundC,Has_roundD,Founded_month
0,CA,San Diego,Bandsintown,1,1/1/2007,2.2493,3.0027,3,375000,music,0,1,0,0,0,0,1
1,CA,San Diego,Plixi,1,3/18/2009,1.0329,1.0329,1,2600000,web,0,0,1,0,0,0,3
2,CA,San Francisco,Inhale Digital,0,8/1/2010,0.0000,1.6685,2,1300000,games_video,1,1,0,0,0,0,8
3,CA,Mountain View,Matisse Networks,0,1/1/2002,4.5452,4.5452,1,7500000,network_hosting,0,0,0,1,0,0,1
4,CA,Mountain View,RingCube Technologies,1,1/1/2005,1.7205,5.2110,3,26000000,software,1,0,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,NY,New York,TRAFFIQ,1,9/6/2006,0.0000,3.9288,3,18000000,advertising,1,0,1,1,0,0,9
505,CA,San Francisco,CoTweet,1,1/1/2009,0.5178,0.5178,1,1100000,advertising,0,0,1,0,0,0,1
506,CA,Sunnyvale,Paracor Medical,0,1/1/1999,8.4959,8.4959,1,44000000,biotech,0,0,0,0,0,1,1
507,CA,San Francisco,Causata,1,1/1/2009,0.7589,2.8329,2,15500000,software,0,0,1,1,0,0,1


In [318]:
ver2['Founded_quarter'] = ""
for l in range(len(ver2['Founded_at'])):
    if int(ver2['Founded_month'][l]) <= 3:
        ver2['Founded_quarter'][l] = str(ver2['Founded_at'][l][-4:]) + ('-Q1')
    elif 6 >= int(ver2['Founded_month'][l]) > 3:
        ver2['Founded_quarter'][l] = str(ver2['Founded_at'][l][-4:]) + ('-Q2')
    elif 9 >= int(ver2['Founded_month'][l]) > 6:
        ver2['Founded_quarter'][l] = str(ver2['Founded_at'][l][-4:]) + ('-Q3')
    else:
        ver2['Founded_quarter'][l] = str(ver2['Founded_at'][l][-4:]) + ('-Q4')
ver2

C:\Users\USER\AppData\Local\Temp/ipykernel_11988/2254215946.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ver2['Founded_quarter'][l] = str(ver2['Founded_at'][l][-4:]) + ('-Q1')
C:\Users\USER\AppData\Local\Temp/ipykernel_11988/2254215946.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ver2['Founded_quarter'][l] = str(ver2['Founded_at'][l][-4:]) + ('-Q3')
C:\Users\USER\AppData\Local\Temp/ipykernel_11988/2254215946.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

,State_code,city,Startup_name,Status,Founded_at,Age_first_funding_year,Age_last_funding_year,Funding_rounds,Funding_total_usd,Category,Has_VC,Has_angel,Has_roundA,Has_roundB,Has_roundC,Has_roundD,Founded_month,Founded_quarter
0,CA,San Diego,Bandsintown,1,1/1/2007,2.2493,3.0027,3,375000,music,0,1,0,0,0,0,1,2007-Q1
1,CA,San Diego,Plixi,1,3/18/2009,1.0329,1.0329,1,2600000,web,0,0,1,0,0,0,3,2009-Q1
2,CA,San Francisco,Inhale Digital,0,8/1/2010,0.0000,1.6685,2,1300000,games_video,1,1,0,0,0,0,8,2010-Q3
3,CA,Mountain View,Matisse Networks,0,1/1/2002,4.5452,4.5452,1,7500000,network_hosting,0,0,0,1,0,0,1,2002-Q1
4,CA,Mountain View,RingCube Technologies,1,1/1/2005,1.7205,5.2110,3,26000000,software,1,0,1,1,0,0,1,2005-Q1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,NY,New York,TRAFFIQ,1,9/6/2006,0.0000,3.9288,3,18000000,advertising,1,0,1,1,0,0,9,2006-Q3
505,CA,San Francisco,CoTweet,1,1/1/2009,0.5178,0.5178,1,1100000,advertising,0,0,1,0,0,0,1,2009-Q1
506,CA,Sunnyvale,Paracor Medical,0,1/1/1999,8.4959,8.4959,1,44000000,biotech,0,0,0,0,0,1,1,1999-Q1
507,CA,San Francisco,Causata,1,1/1/2009,0.7589,2.8329,2,15500000,software,0,0,1,1,0,0,1,2009-Q1


In [319]:
for l in range(len(ver2['Founded_at'])):
    if len(ver2['Founded_month'][l]) == 1:
        ver2['Founded_at'][l] = '0' + str(ver2['Founded_at'][l])
for l in range(len(ver2['Founded_at'])):
    if len(ver2['Founded_at'][l]) == 9:
        ver2['Founded_at'][l] = str(ver2['Founded_at'][l])[:3] + '0' + str(ver2['Founded_at'][l])[3:]
ver2

C:\Users\USER\AppData\Local\Temp/ipykernel_11988/1293631783.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ver2['Founded_at'][l] = '0' + str(ver2['Founded_at'][l])
C:\Users\USER\AppData\Local\Temp/ipykernel_11988/1293631783.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ver2['Founded_at'][l] = str(ver2['Founded_at'][l])[:3] + '0' + str(ver2['Founded_at'][l])[3:]


,State_code,city,Startup_name,Status,Founded_at,Age_first_funding_year,Age_last_funding_year,Funding_rounds,Funding_total_usd,Category,Has_VC,Has_angel,Has_roundA,Has_roundB,Has_roundC,Has_roundD,Founded_month,Founded_quarter
0,CA,San Diego,Bandsintown,1,01/01/2007,2.2493,3.0027,3,375000,music,0,1,0,0,0,0,1,2007-Q1
1,CA,San Diego,Plixi,1,03/18/2009,1.0329,1.0329,1,2600000,web,0,0,1,0,0,0,3,2009-Q1
2,CA,San Francisco,Inhale Digital,0,08/01/2010,0.0000,1.6685,2,1300000,games_video,1,1,0,0,0,0,8,2010-Q3
3,CA,Mountain View,Matisse Networks,0,01/01/2002,4.5452,4.5452,1,7500000,network_hosting,0,0,0,1,0,0,1,2002-Q1
4,CA,Mountain View,RingCube Technologies,1,01/01/2005,1.7205,5.2110,3,26000000,software,1,0,1,1,0,0,1,2005-Q1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,NY,New York,TRAFFIQ,1,09/06/2006,0.0000,3.9288,3,18000000,advertising,1,0,1,1,0,0,9,2006-Q3
505,CA,San Francisco,CoTweet,1,01/01/2009,0.5178,0.5178,1,1100000,advertising,0,0,1,0,0,0,1,2009-Q1
506,CA,Sunnyvale,Paracor Medical,0,01/01/1999,8.4959,8.4959,1,44000000,biotech,0,0,0,0,0,1,1,1999-Q1
507,CA,San Francisco,Causata,1,01/01/2009,0.7589,2.8329,2,15500000,software,0,0,1,1,0,0,1,2009-Q1


In [320]:
def normalizeDate(date):
    newDate = '/'.join(str.zfill(elem,2) for elem in date.split('-'))
    date_obj = datetime.strptime(newDate, '%m/%d/%Y').date()
    return date_obj
ver2['date'] = ""
for l in range(len(ver2['Founded_at'])):
    d = normalizeDate(ver2['Founded_at'][l])
    ver2['date'][l] = d.strftime('%Y/%m/%d')
ver2.drop(labels = 'Founded_at', axis = 1, inplace = True)
ver2 = ver2.rename(columns = {'date': 'Founded_at', 'city': 'City'})
ver2

C:\Users\USER\AppData\Local\Temp/ipykernel_11988/1450621265.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ver2['date'][l] = d.strftime('%Y/%m/%d')


,State_code,City,Startup_name,Status,Age_first_funding_year,Age_last_funding_year,Funding_rounds,Funding_total_usd,Category,Has_VC,Has_angel,Has_roundA,Has_roundB,Has_roundC,Has_roundD,Founded_month,Founded_quarter,Founded_at
0,CA,San Diego,Bandsintown,1,2.2493,3.0027,3,375000,music,0,1,0,0,0,0,1,2007-Q1,2007/01/01
1,CA,San Diego,Plixi,1,1.0329,1.0329,1,2600000,web,0,0,1,0,0,0,3,2009-Q1,2009/03/18
2,CA,San Francisco,Inhale Digital,0,0.0000,1.6685,2,1300000,games_video,1,1,0,0,0,0,8,2010-Q3,2010/08/01
3,CA,Mountain View,Matisse Networks,0,4.5452,4.5452,1,7500000,network_hosting,0,0,0,1,0,0,1,2002-Q1,2002/01/01
4,CA,Mountain View,RingCube Technologies,1,1.7205,5.2110,3,26000000,software,1,0,1,1,0,0,1,2005-Q1,2005/01/01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,NY,New York,TRAFFIQ,1,0.0000,3.9288,3,18000000,advertising,1,0,1,1,0,0,9,2006-Q3,2006/09/06
505,CA,San Francisco,CoTweet,1,0.5178,0.5178,1,1100000,advertising,0,0,1,0,0,0,1,2009-Q1,2009/01/01
506,CA,Sunnyvale,Paracor Medical,0,8.4959,8.4959,1,44000000,biotech,0,0,0,0,0,1,1,1999-Q1,1999/01/01
507,CA,San Francisco,Causata,1,0.7589,2.8329,2,15500000,software,0,0,1,1,0,0,1,2009-Q1,2009/01/01


In [321]:
small = []
for l in range(len(ver2['Age_first_funding_year'])):
    if int(ver2['Age_first_funding_year'][l]) < 0:
        small.append(l)
    elif int(ver2['Age_last_funding_year'][l]) < 0:
        small.append(l)
small

[13, 131, 268, 306]

In [322]:
for s in small:
    ver2.drop(labels = s, axis = 0, inplace = True)

In [323]:
ver2 = ver2.reset_index()
ver2.drop(labels = 'index', axis = 1, inplace = True)
ver2

,State_code,City,Startup_name,Status,Age_first_funding_year,Age_last_funding_year,Funding_rounds,Funding_total_usd,Category,Has_VC,Has_angel,Has_roundA,Has_roundB,Has_roundC,Has_roundD,Founded_month,Founded_quarter,Founded_at
0,CA,San Diego,Bandsintown,1,2.2493,3.0027,3,375000,music,0,1,0,0,0,0,1,2007-Q1,2007/01/01
1,CA,San Diego,Plixi,1,1.0329,1.0329,1,2600000,web,0,0,1,0,0,0,3,2009-Q1,2009/03/18
2,CA,San Francisco,Inhale Digital,0,0.0000,1.6685,2,1300000,games_video,1,1,0,0,0,0,8,2010-Q3,2010/08/01
3,CA,Mountain View,Matisse Networks,0,4.5452,4.5452,1,7500000,network_hosting,0,0,0,1,0,0,1,2002-Q1,2002/01/01
4,CA,Mountain View,RingCube Technologies,1,1.7205,5.2110,3,26000000,software,1,0,1,1,0,0,1,2005-Q1,2005/01/01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,NY,New York,TRAFFIQ,1,0.0000,3.9288,3,18000000,advertising,1,0,1,1,0,0,9,2006-Q3,2006/09/06
501,CA,San Francisco,CoTweet,1,0.5178,0.5178,1,1100000,advertising,0,0,1,0,0,0,1,2009-Q1,2009/01/01
502,CA,Sunnyvale,Paracor Medical,0,8.4959,8.4959,1,44000000,biotech,0,0,0,0,0,1,1,1999-Q1,1999/01/01
503,CA,San Francisco,Causata,1,0.7589,2.8329,2,15500000,software,0,0,1,1,0,0,1,2009-Q1,2009/01/01


In [324]:
final = ver2.copy()
for l in range(len(final['Category'])):
    if final['Category'][l] == 'network_hosting':
        final['Category'][l] = 'web'
    elif final['Category'][l] == 'software':
        final['Category'][l] = 'hardware&software'
    elif final['Category'][l] == 'games_video':
        final['Category'][l] = 'games'
    elif final['Category'][l] == 'hardware':
        final['Category'][l] = 'hardware&software'
    elif final['Category'][l] == 'ecommerce':
        final['Category'][l] = 'e-commerce'
    elif final['Category'][l] == 'semiconductor':
        final['Category'][l] = 'semiconductors'
    elif final['Category'][l] == 'messaging':
        final['Category'][l] = 'hardware&software'
    elif final['Category'][l] == 'cleantech':
        final['Category'][l] = 'clean_technology'
    elif final['Category'][l] == 'news':
        final['Category'][l] = 'social_media'
    elif final['Category'][l] == 'social':
        final['Category'][l] = 'social_media'
    elif final['Category'][l] == 'photo_video':
        final['Category'][l] = 'social_media'
    elif final['Category'][l] == 'music':
        final['Category'][l] = 'art&music'
    elif final['Category'][l] == 'health':
        final['Category'][l] = 'health&medical'
    elif final['Category'][l] == 'medical':
        final['Category'][l] = 'health&medical'
    elif final['Category'][l] == 'real_estate':
        final['Category'][l] = 'finance'
    elif final['Category'][l] == 'biotech':
        final['Category'][l] = 'biotechnology'
final.Category.value_counts()

C:\Users\USER\AppData\Local\Temp/ipykernel_11988/1068256099.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['Category'][l] = 'art&music'
C:\Users\USER\AppData\Local\Temp/ipykernel_11988/1068256099.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['Category'][l] = 'games'
C:\Users\USER\AppData\Local\Temp/ipykernel_11988/1068256099.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['Category'][l] = 'web'
C:\User

web                  105
hardware&software    100
mobile                43
advertising           39
enterprise            39
games                 36
social_media          18
e-commerce            14
biotechnology         14
semiconductors        13
analytics             13
security              11
search                 9
public_relations       8
clean_technology       8
fashion                6
travel                 5
finance                5
other                  4
health&medical         4
art&music              4
education              3
automotive             1
sports                 1
hospitality            1
transportation         1
Name: Category, dtype: int64

In [327]:
value_to_drop = ['education', 'automotive', 'hospitality', 'transportation', 'other', 'sports', 'search', 'enterprise']
for d in value_to_drop:
    final = final[final['Category'] != d]
for d in value_to_drop:
    final = final[final['Category'] != d]

In [328]:
final = final.reset_index()
final = final[:423]
final.drop(labels = 'index', axis = 1, inplace = True)
for l in range(len(final['Age_first_funding_year'])):
    final['Age_first_funding_year'][l] = int(round(int(final['Age_first_funding_year'][l])))
    final['Age_last_funding_year'][l] = int(round(int(final['Age_last_funding_year'][l])))
final.to_csv('./small.csv')

C:\Users\USER\AppData\Local\Temp/ipykernel_11988/1745379674.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['Age_first_funding_year'][l] = int(round(int(final['Age_first_funding_year'][l])))
C:\Users\USER\AppData\Local\Temp/ipykernel_11988/1745379674.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['Age_last_funding_year'][l] = int(round(int(final['Age_last_funding_year'][l])))
